In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import DataFunctions as data

2024-06-11 09:31:59.310489: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 09:31:59.343474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 09:31:59.343494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 09:31:59.344449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 09:31:59.350105: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Se obtienen los datos de los sensores y las etiquetas deducidas
print("Obteniendo datos...")
data_2206_17_bag, tags_220617_bag = data.get_data('/home/cmonedero/TFG/User1/Motion/User1_220617_Bag.csv', '/home/cmonedero/TFG/User1/220617/Label_New_User1_220617.txt')
print(len(data_2206_17_bag))
print(len(tags_220617_bag))


Obteniendo datos...


Hecho!

1825611
1825611


In [3]:
print(data_2206_17_bag[['Módulo Acelerómetro', 'Módulo Giroscopio', 'Módulo Magnetómetro', 'Módulo Orientación', 'Módulo Gravedad', 'Módulo Acel. Lineal']].to_numpy())

[[ 1.07019924  1.63547533 -0.49684637  0.02752316  0.02210398  1.90782759]
 [ 1.50750962  1.216529   -0.47890487  0.02751279  0.02248623  1.67817769]
 [ 1.66881362  1.40994663 -0.46373208  0.02751719  0.0222481   2.34294079]
 ...
 [-0.04260902 -0.17239478 -0.67691574  0.02764215  0.02275917 -0.24600538]
 [-0.05522627 -0.16790638 -0.67455615  0.02763254  0.02263291 -0.18964522]
 [-0.22125715 -0.17316984 -0.66096158  0.02762491  0.02281223 -0.21663067]]


In [4]:
print("Dividiendo en ventanas el dataset...")
x_windows_220617_bag, y_windows_220617_bag = data.divide_data_arrays_into_windows(data_2206_17_bag, tags_220617_bag, moda=False, window_size=500)

Dividiendo en ventanas el dataset...
Hecho!



In [5]:
print("Tamaño ventanas X --> ", x_windows_220617_bag.shape)
print("Tamaño ventanas Y --> ", y_windows_220617_bag.shape)

Tamaño ventanas X -->  (3651, 500, 6)
Tamaño ventanas Y -->  (3651, 1)


In [7]:
#Se divide el dataset en los subconjuntos de train y test
print("Dividiendo en train y test...")
x_train_220617_bag, y_train_220617_bag, x_test_220617_bag, y_test_220617_bag = data.divide_train_test(x_windows_220617_bag, y_windows_220617_bag, 0.7)
print("Tamaño x_train --> ", x_train_220617_bag.shape)
print("Tamaño y_train --> ", y_train_220617_bag.shape)
print("Tamaño x_test --> ", x_test_220617_bag.shape)
print("Tamaño y_test -->", y_test_220617_bag.shape)

Dividiendo en train y test...
Tamaño x_train -->  (2555, 500, 6)
Tamaño y_train -->  (2555, 1)
Tamaño x_test -->  (1096, 500, 6)
Tamaño y_test --> (1096, 1)


In [8]:
input_shape = x_train_220617_bag.shape[1:]
print(input_shape)
print(x_train_220617_bag.shape)

(500, 6)
(2555, 500, 6)


In [9]:
def multiheadattention(inputs, head_size, num_heads, ff_dim, dropout=0, i=0):

    # print("Bloque número ", i, " --> inputs shape ", inputs)
    x = BatchNormalization(epsilon=1e-6)(inputs)
    # print("Después de la normalización por batches ---> ", x)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout )(x, x)
    # print("Después de la capa multi atención --> ", x)
    x = Dropout(dropout)(x)
    # print("Después del dropout --> ", x)
    res = x + inputs
    # print("Bloque residual --> ", res)

    return x + res

In [10]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    i = 0
    for _ in range(num_blocks):
        x = multiheadattention(x, head_size, num_heads, ff_dim, dropout, i)
        i += 1
    # print("\n\n")
    print("Después de las capas de atención --> ", x)
    x = GlobalAveragePooling1D()(x)
    print("Despues del global average pooling --> ", x)
    # x = Dense(input_shape[1], activation='relu')(x) 
    print("Despues de la capa FC --> ", x)
    # x = Dropout(mlp_dropout)(x)
    print("Despues del dropout --> ", x)
    outputs = Dense(8, 'softmax')(x)
    print("\n\nSalida --> ", outputs)
    

    model = keras.Model(inputs, outputs)

    return keras.Model(inputs, outputs)

In [11]:
input_shape = x_train_220617_bag.shape[1:]
model_220617_bag = build_model(input_shape, head_size=4, num_heads=8, ff_dim=4, mlp_units=[5], num_blocks=5, mlp_dropout=0.4, dropout=0.25)
checkpoint_filepath = '/home/cmonedero/TFG/MHA/checkpoint.model.keras'
cp_220617_bag = ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, save_freq='epoch')

print("Entrenando el modelo...")
model_220617_bag.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model_220617_bag.fit(tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float64), epochs=10, callbacks=cp_220617_bag, validation_split=0.1)

2024-05-15 16:29:30.378858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 41198 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6
2024-05-15 16:29:30.381108: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 41198 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:98:00.0, compute capability: 8.6


Después de las capas de atención -->  KerasTensor(type_spec=TensorSpec(shape=(None, 500, 6), dtype=tf.float32, name=None), name='tf.__operators__.add_9/AddV2:0', description="created by layer 'tf.__operators__.add_9'")
Despues del global average pooling -->  KerasTensor(type_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name=None), name='global_average_pooling1d/Mean:0', description="created by layer 'global_average_pooling1d'")
Despues de la capa FC -->  KerasTensor(type_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name=None), name='global_average_pooling1d/Mean:0', description="created by layer 'global_average_pooling1d'")
Despues del dropout -->  KerasTensor(type_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name=None), name='global_average_pooling1d/Mean:0', description="created by layer 'global_average_pooling1d'")


Salida -->  KerasTensor(type_spec=TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), name='dense/Softmax:0', description="created by layer 'den

2024-05-15 16:29:34.261151: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-15 16:29:35.647745: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd77afd4850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-15 16:29:35.647782: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-05-15 16:29:35.647788: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA A40, Compute Capability 8.6
2024-05-15 16:29:35.655605: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-15 16:29:35.702273: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1715790575.831357  984951 device_compiler.h:186] Compiled clust

72/72 [==============================] - 15s 99ms/step - loss: 1.2881 - accuracy: 0.5629 - val_loss: 0.3702 - val_accuracy: 0.9375
Epoch 2/10
72/72 [==============================] - 6s 90ms/step - loss: 0.6497 - accuracy: 0.7864 - val_loss: 0.3948 - val_accuracy: 0.8594
Epoch 3/10
72/72 [==============================] - 7s 92ms/step - loss: 0.4798 - accuracy: 0.8556 - val_loss: 0.2964 - val_accuracy: 0.9258
Epoch 4/10
72/72 [==============================] - 7s 91ms/step - loss: 0.4066 - accuracy: 0.8512 - val_loss: 0.2864 - val_accuracy: 0.9219
Epoch 5/10
72/72 [==============================] - 7s 92ms/step - loss: 0.3473 - accuracy: 0.8765 - val_loss: 0.1767 - val_accuracy: 0.9570
Epoch 6/10
72/72 [==============================] - 6s 89ms/step - loss: 0.3077 - accuracy: 0.8952 - val_loss: 0.2655 - val_accuracy: 0.9375
Epoch 7/10
72/72 [==============================] - 7s 92ms/step - loss: 0.2685 - accuracy: 0.8943 - val_loss: 0.1405 - val_accuracy: 0.9648
Epoch 8/10
72/72 [=====

In [12]:
evals = model_220617_bag.evaluate(tf.constant(x_test_220617_bag), tf.cast(tf.constant(y_test_220617_bag), tf.float64))

35/35 [==============================] - 1s 27ms/step - loss: 20.4949 - accuracy: 0.7308


In [13]:
model_220617_bag.save('MHAMemo/mha.h5')

/home/cmonedero/TFG/tfg_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
